In [1]:
import sys
print(sys.executable)


c:\projects\retail-analytics-dashboard\retail_env\Scripts\python.exe


In [2]:
# Cell 1: Verify data exists
import os
import pandas as pd

file_path = '../data/processed/clean_data.csv'

if os.path.exists(file_path):
    print(" Data file found!")
    df = pd.read_csv(file_path)
    print(f" Loaded {len(df):,} rows")
else:
    print(" Data file not found. Run scripts/01_data_ingestion.py first!")

 Data file found!
 Loaded 397,884 rows


In [3]:
# Cell 1: Setup
"""
Exploratory Data Analysis - Retail Dataset
Goal: Understand data patterns, identify insights for dashboard
"""

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

# Styling
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports complete")

✓ Imports complete


In [4]:
# Cell 2: Load Data
df = pd.read_csv('../data/processed/clean_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

Dataset shape: (397884, 8)

Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

First few rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [5]:
# Cell 3: Quick Stats
print("=== Dataset Overview ===\n")
print(df.info())
print("\n=== Statistical Summary ===\n")
print(df.describe())

=== Dataset Overview ===

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397884 non-null  int64  
 1   StockCode    397884 non-null  object 
 2   Description  397884 non-null  object 
 3   Quantity     397884 non-null  int64  
 4   InvoiceDate  397884 non-null  object 
 5   UnitPrice    397884 non-null  float64
 6   CustomerID   397884 non-null  float64
 7   Country      397884 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 24.3+ MB
None

=== Statistical Summary ===

           InvoiceNo       Quantity      UnitPrice     CustomerID
count  397884.000000  397884.000000  397884.000000  397884.000000
mean   560616.934451      12.988238       3.116488   15294.423453
std     13106.117773     179.331775      22.097877    1713.141560
min    536365.000000       1.000000       0.001000   12346.000000
25%   

In [6]:
# Cell 4: Data Preparation
# Convert date column to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Create useful columns
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.day_name()
df['Hour'] = df['InvoiceDate'].dt.hour

# Calculate total sales
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

print("✓ Feature engineering complete")
df.head()

✓ Feature engineering complete


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,DayOfWeek,Hour,TotalSales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,Wednesday,8,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,Wednesday,8,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,12,Wednesday,8,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,Wednesday,8,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,Wednesday,8,20.34


In [7]:
# Cell 5: Key Metrics Overview
total_revenue = df['TotalSales'].sum()
total_transactions = df['InvoiceNo'].nunique()
total_customers = df['CustomerID'].nunique()
total_products = df['StockCode'].nunique()
avg_order_value = total_revenue / total_transactions

print("=== Business Metrics ===")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Total Transactions: {total_transactions:,}")
print(f"Total Customers: {total_customers:,}")
print(f"Total Products: {total_products:,}")
print(f"Average Order Value: ${avg_order_value:.2f}")

=== Business Metrics ===
Total Revenue: $8,911,407.90
Total Transactions: 18,532
Total Customers: 4,338
Total Products: 3,665
Average Order Value: $480.87


In [8]:
# Cell 6: Time Series Analysis
# Daily sales trend
daily_sales = df.groupby(df['InvoiceDate'].dt.date)['TotalSales'].sum().reset_index()
daily_sales.columns = ['Date', 'Sales']

fig = px.line(daily_sales, x='Date', y='Sales', 
              title='Daily Sales Trend',
              labels={'Sales': 'Total Sales ($)'})
fig.update_layout(height=500)
fig.show()

# Key insight
print("\n INSIGHT: Look for trends, seasonality, anomalies")


 INSIGHT: Look for trends, seasonality, anomalies


In [9]:
# Cell 7: Sales by Day of Week
day_sales = df.groupby('DayOfWeek')['TotalSales'].sum().reset_index()

# Order days properly
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_sales['DayOfWeek'] = pd.Categorical(day_sales['DayOfWeek'], categories=day_order, ordered=True)
day_sales = day_sales.sort_values('DayOfWeek')

fig = px.bar(day_sales, x='DayOfWeek', y='TotalSales',
             title='Sales by Day of Week',
             labels={'TotalSales': 'Total Sales ($)'})
fig.show()

print("\n INSIGHT: Which days are strongest? Weekend vs weekday patterns?")


 INSIGHT: Which days are strongest? Weekend vs weekday patterns?


In [10]:
# Cell 8: Top Products Analysis
top_products = df.groupby('Description')['TotalSales'].sum().nlargest(10).reset_index()

fig = px.bar(top_products, x='TotalSales', y='Description', 
             orientation='h',
             title='Top 10 Products by Revenue')
fig.show()

print("\n INSIGHT: Which products drive most revenue?")


 INSIGHT: Which products drive most revenue?


In [11]:
# Cell 9: Customer Analysis
customer_sales = df.groupby('CustomerID')['TotalSales'].sum().reset_index()
customer_sales = customer_sales.sort_values('TotalSales', ascending=False)

print("=== Customer Insights ===")
print(f"Top 10% customers contribute: {customer_sales.head(int(len(customer_sales)*0.1))['TotalSales'].sum() / total_revenue * 100:.1f}% of revenue")
print(f"Average customer value: ${customer_sales['TotalSales'].mean():.2f}")

# Distribution plot
fig = px.histogram(customer_sales, x='TotalSales', nbins=50,
                   title='Customer Value Distribution')
fig.show()

print("\n INSIGHT: Is this a power law distribution? Few customers = most revenue?")

=== Customer Insights ===
Top 10% customers contribute: 61.3% of revenue
Average customer value: $2054.27



 INSIGHT: Is this a power law distribution? Few customers = most revenue?


In [13]:
# Cell 10: Seasonality Check
monthly_sales = df.groupby(['Year', 'Month'])['TotalSales'].sum().reset_index()
monthly_sales['YearMonth'] = monthly_sales['Year'].astype(str) + '-' + monthly_sales['Month'].astype(str).str.zfill(2)

fig = px.line(monthly_sales, x='YearMonth', y='TotalSales',
              title='Monthly Sales Trend',
              markers=True)
fig.show()

print("\n INSIGHT: Any seasonal patterns? Holiday spikes?")


 INSIGHT: Any seasonal patterns? Holiday spikes?


In [14]:
# Cell 11: Summary of Key Insights

insights = f"""
=== KEY INSIGHTS FOR DASHBOARD ===

1. REVENUE: ${total_revenue:,.2f} from {total_transactions:,} transactions

2. CUSTOMER BASE: {total_customers:,} unique customers
   - Top 10% contribute: [calculate]% of revenue
   - Average customer value: ${customer_sales['TotalSales'].mean():.2f}

3. PRODUCT PERFORMANCE:
   - Top product: {top_products.iloc[0]['Description']}
   - Top 10 products account for: [calculate]% of revenue

4. TEMPORAL PATTERNS:
   - Strongest day: [identify from day_sales]
   - Seasonal trend: [identify from monthly chart]

5. AVERAGE ORDER VALUE: ${avg_order_value:.2f}

6. BUSINESS HEALTH:
   - [Growing/Stable/Declining] based on time series
   - [High/Low] customer concentration risk
"""

print(insights)

# Save these insights for later
with open('../docs/initial_insights.md', 'w') as f:
    f.write(insights)

print("\n Analysis complete! Insights saved to docs/initial_insights.md")


=== KEY INSIGHTS FOR DASHBOARD ===

1. REVENUE: $8,911,407.90 from 18,532 transactions

2. CUSTOMER BASE: 4,338 unique customers
   - Top 10% contribute: [calculate]% of revenue
   - Average customer value: $2054.27

3. PRODUCT PERFORMANCE:
   - Top product: PAPER CRAFT , LITTLE BIRDIE
   - Top 10 products account for: [calculate]% of revenue

4. TEMPORAL PATTERNS:
   - Strongest day: [identify from day_sales]
   - Seasonal trend: [identify from monthly chart]

5. AVERAGE ORDER VALUE: $480.87

6. BUSINESS HEALTH:
   - [Growing/Stable/Declining] based on time series
   - [High/Low] customer concentration risk


 Analysis complete! Insights saved to docs/initial_insights.md
